In [1]:
import tkinter as tk
from scipy.io import wavfile
import pyaudio
import threading
import numpy as np

#### Import Track

In [2]:
sample_rate, audio_data = wavfile.read("120 BPM - ROCK - 4⧸4 Drum Track - Metronome - Drum Beat.wav")
audio_data = audio_data.T
print("sample_rate :",sample_rate)

sample_rate : 48000


In [3]:
#low shelf filter
def calc_low_shelf_tf(dB_Gain, sample_rate):
    S =1
    f0 = 500
    w0 = np.round(f0/sample_rate, 2)                                              # cut off
    A = 10**(dB_Gain/40)
    sqrt_A = np.sqrt(A)
    sin = np.sin(w0)
    cos = np.cos(w0)
    alpha = sin*0.5*np.sqrt((A+1/A)*(1/S - 1)+2)
    #
    b0 = A*((A+1) + (A-1)*cos + 2*sqrt_A*alpha )
    b1 = -2*A*( (A-1) + (A+1)*cos)
    b2 = A*((A+1) + (A-1)*cos - 2*sqrt_A*alpha )
    #
    a0 = ( (A+1) - (A-1)*cos + 2*sqrt_A*alpha )
    a1 = 2*( (A-1) - (A+1)*cos)
    a2 = ( (A+1) - (A-1)*cos - 2*sqrt_A*alpha )
    deno = [a0, a1, a2]
    nume = [b0, b1, b2]
    return deno, nume

#high shelf filter
def calc_high_shelf_tf(dB_Gain, sample_rate):
    S =1
    f0 = 8000
    w0 = np.round(f0/sample_rate, 2)                                              # cut off
    A = 10**(dB_Gain/40)
    sqrt_A = np.sqrt(A)
    sin = np.sin(w0)
    cos = np.cos(w0)
    alpha = sin*0.5*np.sqrt((A+1/A)*(1/S - 1)+2)
    #
    b0 = A*( (A+1) - (A-1)*cos + 2*sqrt_A*alpha )
    b1 = 2*A*( (A-1) - (A+1)*cos)
    b2 = A*( (A+1) - (A-1)*cos - 2*sqrt_A*alpha )
    #
    a0 = (A+1) + (A-1)*cos + 2*sqrt_A*alpha 
    a1 = -2*( (A-1) + (A+1)*cos)
    a2 = (A+1) + (A-1)*cos - 2*sqrt_A*alpha 
    deno = [a0, a1, a2]
    nume = [b0, b1, b2]
    return deno, nume

#peaking EQ filter
def calc_peaking_eq_tf(dBGain, f0, sample_rate):
    #S = 7.5*(10**-4)*f0
    #S = 1
    Q = 1/np.sqrt(2)
    w0 = np.round(f0/sample_rate, 2)                                              # cut off
    A = 10**(dBGain/40)
    sqrt_A = np.sqrt(A)
    sin = np.sin(w0)
    cos = np.cos(w0)
    #alpha = sin*0.5*np.sqrt((A+1/A)*(1/S - 1)+2)
    alpha = sin/(2*Q)
    #
    b0 = 1 + alpha*A
    b1 = -2*cos
    b2 = 1 - alpha*A
    #
    a0 = 1 + alpha/A 
    a1 = -2*cos
    a2 = 1 - alpha/A
    nume = [a0, a1, a2]
    deno = [b0, b1, b2]
    return deno, nume

#differential equation
def shelf_filter(input_array, prev_output, deno, nume):
    # input_array - input audio data
    # prev_output - previous last two outputs 
    # deno - transfer function denominator
    # nume - transfer function neumarator
    a0, a1, a2 = deno       
    b0, b1, b2 = nume       
    #Input
    x = np.concatenate((np.zeros(2), input_array))
    y = np.concatenate((prev_output, np.zeros(len(input_array))))
    #Calculate y
    for k in range(2,len(y)):
        #if y[k-1]> 2**30:
         #   y[k-1] = 0
         #   y[k-2] = 0
        x_terms = a0*x[k] + a1*x[k-1] + a2*x[k-2]
        y_terms = -b1*y[k-1] - b2*y[k-2]
        #y_terms = 0
        #print('y_terms :',y_terms)
        #print('y[k-1] :',y[k-1])
        #print('y[k-2] :',y[k-2])
        y[k] = (x_terms + y_terms)/b0
    y_prev = y[0:2]
    return y, y_prev

# scale output
def scale_array(input_array, output_array):
    #input
    input_max_size = 2**((input_array.dtype.itemsize)*8 - 1)
    input_max = np.max(input_array)
    input_ratio = input_max / input_max_size
    #output
    output_max_size = 2**((output_array.dtype.itemsize)*8 - 1)
    output_max = np.max(output_array)
    output_ratio = (1.3*output_max) / output_max_size
    #scaling
    scaling_factor = int(input_ratio/output_ratio)
    return output_array*scaling_factor

In [4]:
def update_transfer(value, low = False, mid = False, high=False, fre = False):
    global deno_L, nume_L, deno_M, nume_M, deno_H, nume_H, sample_rate, f0, mid_gain
    value = int(value)
    if low:
        deno_L, nume_L = calc_low_shelf_tf(value, sample_rate)
    elif high:
        deno_H, nume_H = calc_high_shelf_tf(value, sample_rate)
    elif mid:
        mid_gain = value
        deno_M, nume_M = calc_peaking_eq_tf(value, f0, sample_rate)
    elif fre:
        f0 = value
        deno_M, nume_M = calc_peaking_eq_tf(mid_gain, -value, sample_rate)

In [5]:
def create_play():
    thread = threading.Thread(target=play)
    thread.start()

def play():
    global audio_data, sample_rate
    global deno_L, nume_L, deno_M, nume_M, deno_H, nume_H 
    
    CHUNK = sample_rate
    p = pyaudio.PyAudio()
    stream = p.open(format=pyaudio.paInt32,
                channels = 1,
                rate = sample_rate,
                frames_per_buffer = CHUNK,
                output = True)
    
    y_prev = np.zeros(2)
    for k in range(4*CHUNK, 40*CHUNK, CHUNK):
        x = np.array(audio_data[0, k : k+CHUNK])
        #Low Shelf Filter
        y, y_prev = shelf_filter(x, y_prev, deno_L, nume_L)
        #High Shelf Filter
        y, y_prev = shelf_filter(y, y_prev, deno_H, nume_H)
        #High Shelf Filter
        y, y_prev = shelf_filter(y, y_prev, deno_M, nume_M)
        #Scaling
        y_n = y.astype(int)
        y_n = scale_array(x, y_n)
        #Reshaping
        y_n = y_n.reshape(1, -1)
        #Playing
        stream.write(y_n.tobytes()) 

In [11]:
deno_L, nume_L = calc_low_shelf_tf(0, sample_rate)
f0 = 10000
mid_gain = 0
deno_M, nume_M = calc_peaking_eq_tf(mid_gain, f0, sample_rate)
deno_H, nume_H = calc_high_shelf_tf(0, sample_rate)


# Create the main window
root = tk.Tk()
root.title("EQ")

# Create a label to display the selected value
label = tk.Label(root, text="Equalizer")
label.pack(pady=10)

root.geometry("500x400")

# Bass vertical slider 
bass_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value: update_transfer(value, low=True))
bass_slider.place(x=50, y=50)
label_s = tk.Label(root, text="LF")
label_s.place(x=65, y=260)

# frequency slider 
frequency_slider = tk.Scale(root, from_=500, to=12000, orient=tk.VERTICAL, length=200, command=lambda value:update_transfer(value, fre = True))
frequency_slider.place(x=350, y=50)
label_s = tk.Label(root, text="Fre")
label_s.place(x=380, y=260)

# Mid vertical slider 
mid_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value:update_transfer(value, mid = True))
mid_slider.place(x=150, y=50)
label_s = tk.Label(root, text="MF")
label_s.place(x=170, y=260)

# Treble vertical slider 
treble_slider = tk.Scale(root, from_=15, to=-15, orient=tk.VERTICAL, length=200, command=lambda value:update_transfer(value, high = True))
treble_slider.place(x=250, y=50)
label_s = tk.Label(root, text="HF")
label_s.place(x=270, y=260)

#play/pause
play_button = tk.Button(text = 'Play/Pause', command = create_play)
play_button.place(x=200, y=360)

# Run the Tkinter event loop
root.mainloop()